In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
# df =pd.read_csv('../../data/more_features/more_feats_correlated_testing_noanem_noisy_6.csv')
# df = df.fillna(-1)
# df.head()
train_df = pd.read_csv('../../data/more_features/train_sets/train_set_noisy_2.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,12.924661,91.609677,-1.000000,-1.000000,471.681802,77.891910,-1.000000,4.977922,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,38.773982,-1.000000,0.0
1,11.831878,-1.000000,3.647624,-1.000000,-1.000000,93.765932,99.218319,3.785558,1,-1.000000,140.666497,123.188606,77.869533,3.685457,-1.000000,35.495634,-1.000000,5.0
2,7.400960,180.293376,4.837502,3.580706,411.766078,99.376432,192.449622,2.234220,0,0.664748,54.775702,124.190833,35.464689,12.388074,-1.000000,22.202879,46.737610,5.0
3,11.217343,70.217945,-1.000000,-1.000000,484.687092,79.211804,50.042720,4.248360,1,1.771297,108.277545,78.201225,13.750807,24.623277,41.191027,33.652028,10.324748,4.0
4,9.384677,30.563795,2.602770,-1.000000,355.383498,100.429776,-1.000000,2.803355,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,28.154031,-1.000000,5.0


In [4]:
test_df = pd.read_csv('../../data/more_features/train_sets/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,-1.000000,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1,-1.000000,36.703810,122.310168,49.897027,20.353251,-1.000000,34.173407,46.631388,6
1,10.355048,272.506966,-1.000000,0.000000,128.706486,101.912313,-1.000000,3.048223,1,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.000000,2
2,11.159385,-1.000000,-1.000000,0.000000,-1.000000,103.395616,77.212369,3.237870,0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.000000,2
3,8.179735,140.876632,5.484515,-1.000000,303.740826,99.877458,155.573175,2.456931,0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,9.916825,-1.000000,3.909810,-1.000000,-1.000000,90.543986,189.347916,3.285748,0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5


In [5]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [6]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [7]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [ ]:
for steps in [int(10e6), int(9.5e6), int(10.5e6)]:
#for steps in [int(7.5e6), int(7.8e6), int(8.3e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/constant_test_set/dqn_noisy_2_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.6      |
|    ep_rew_mean      | -0.92    |
|    exploration_rate | 0.722    |
|    success_rate     | 0.05     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 970      |
|    time_elapsed     | 301      |
|    total_timesteps  | 292842   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 14.4     |
|    n_updates        | 60710    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.44     |
|    ep_rew_mean      | -0.9     |
|    exploration_rate | 0.381    |
|    success_rate     | 0.08     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 810      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.95     |
|    ep_rew_mean      | 0.1      |
|    exploration_rate | 0.05     |
|    success_rate     | 0.59     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 441      |
|    time_elapsed     | 14301    |
|    total_timesteps  | 6314875  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.156    |
|    n_updates        | 1566218  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.04     |
|    ep_rew_mean      | 0.28     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.67     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 448      |
|    time_elapsed     | 14950    |
|    total_timesteps  | 6708733  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.23     |
|    ep_rew_mean      | -0.34    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.36     |
| time/               |          |
|    episodes         | 600000   |
|    fps              | 634      |
|    time_elapsed     | 3678     |
|    total_timesteps  | 2333700  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0189   |
|    n_updates        | 570924   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.17     |
|    ep_rew_mean      | -0.26    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 700000   |
|    fps              | 622      |
|    time_elapsed     | 4430     |
|    total_timesteps  | 2757523  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.88     |
|    ep_rew_mean      | 0.16     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.61     |
| time/               |          |
|    episodes         | 2100000  |
|    fps              | 503      |
|    time_elapsed     | 18131    |
|    total_timesteps  | 9123740  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0177   |
|    n_updates        | 2268434  |
----------------------------------
using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.32     |
|    ep_rew_mean      | -0.88    |
|    exploration_rate | 0.735    |
|    success_rate     | 0.08     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 994      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.74     |
|    ep_rew_mean      | 0.22     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.64     |
| time/               |          |
|    episodes         | 1500000  |
|    fps              | 461      |
|    time_elapsed     | 12737    |
|    total_timesteps  | 5877204  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0933   |
|    n_updates        | 1456800  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.1      |
|    ep_rew_mean      | 0.22     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.65     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 458      |
|    time_elapsed     | 13702    |
|    total_timesteps  | 6283531  |
| train/              |          |
|    learning_rate  

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)